In [1]:
#!pip install graphviz

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.display import SVG
from graphviz import Source
from sklearn.tree import export_text

In [3]:
loan = pd.read_csv("Loan_Data.csv")
loan.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [4]:
loan.isnull().sum()
#loan['Education'] = loan.Education.map({'Not Graduate':0, 'Graduate':1})
#loan.Education.head(10)

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [5]:
loan['Gender'] = loan.Gender.map({'Female':0, 'Male':1})
loan['Married'] = loan.Married.map({'No':0, 'Yes':1})
loan['Education'] = loan.Education.map({'Not Graduate':0, 'Graduate':1})
loan['Self_Employed'] = loan.Self_Employed.map({'No':0, 'Yes':1})
loan['Loan_Status'] = loan.Loan_Status.map({'N':0, 'Y':1})
loan['Dependents'] = loan.Dependents.map({'0':0, '1':1, '2':2, '3+':3})


In [6]:
loan.Married.fillna(0, inplace=True)
loan.Gender.fillna(1, inplace=True)
loan.Self_Employed.fillna(0, inplace=True)
loan.Dependents.fillna(0, inplace=True)
loan.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1.0,0.0,0.0,1,0.0,5849,0.0,NaN,360.0,1.0,Urban,1
1,LP001003,1.0,1.0,1.0,1,0.0,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,1.0,1.0,0.0,1,1.0,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,1.0,1.0,0.0,0,0.0,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,1.0,0.0,0.0,1,0.0,6000,0.0,141.0,360.0,1.0,Urban,1
5,LP001011,1.0,1.0,2.0,1,1.0,5417,4196.0,267.0,360.0,1.0,Urban,1
6,LP001013,1.0,1.0,0.0,0,0.0,2333,1516.0,95.0,360.0,1.0,Urban,1
7,LP001014,1.0,1.0,3.0,1,0.0,3036,2504.0,158.0,360.0,0.0,Semiurban,0
8,LP001018,1.0,1.0,2.0,1,0.0,4006,1526.0,168.0,360.0,1.0,Urban,1
9,LP001020,1.0,1.0,1.0,1,0.0,12841,10968.0,349.0,360.0,1.0,Semiurban,0


In [7]:
feature_cols = ['Gender', 'Married', 'Education', 'Self_Employed', 'ApplicantIncome','Dependents']

# define X and y
X = loan[feature_cols]
y = loan.Loan_Status

In [8]:
# fit a classification tree with max_depth=3 on all data
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

DecisionTreeClassifier(max_depth=3, random_state=1)

In [9]:
# compute the feature importances
pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})

,feature,importance
0,Gender,0.000000
1,Married,0.214441
2,Education,0.156288
3,Self_Employed,0.000000
4,ApplicantIncome,0.508846
5,Dependents,0.120425


In [10]:
# fit a classification tree with max_depth=3 on all data
treeclf2 = DecisionTreeClassifier(max_depth=6, random_state=1)
treeclf2.fit(X, y)

DecisionTreeClassifier(max_depth=6, random_state=1)

In [11]:
# compute the feature importances
pd.DataFrame({'feature':feature_cols, 'importance':treeclf2.feature_importances_})

,feature,importance
0,Gender,0.052486
1,Married,0.085583
2,Education,0.041770
3,Self_Employed,0.000000
4,ApplicantIncome,0.666850
5,Dependents,0.153311


In [12]:
r = export_text(treeclf, feature_names=feature_cols)
>>> print(r)

#graph = Source(export_graphviz(treeclf, out_file=None,
                                    #feature_names=feature_cols,
                                    #class_names=['0', '1'], filled = True))
#display(SVG(graph.pipe(format='svg')))

#with open("loan.dot", 'wb') as f:
  #  f = export_graphviz(treeclf, out_file=f, feature_names=feature_cols)

|--- Education <= 0.50
|   |--- ApplicantIncome <= 1810.00
|   |   |--- Dependents <= 1.00
|   |   |   |--- class: 0
|   |   |--- Dependents >  1.00
|   |   |   |--- class: 1
|   |--- ApplicantIncome >  1810.00
|   |   |--- ApplicantIncome <= 2267.00
|   |   |   |--- class: 1
|   |   |--- ApplicantIncome >  2267.00
|   |   |   |--- class: 1
|--- Education >  0.50
|   |--- Married <= 0.50
|   |   |--- ApplicantIncome <= 2457.00
|   |   |   |--- class: 0
|   |   |--- ApplicantIncome >  2457.00
|   |   |   |--- class: 1
|   |--- Married >  0.50
|   |   |--- ApplicantIncome <= 412.00
|   |   |   |--- class: 0
|   |   |--- ApplicantIncome >  412.00
|   |   |   |--- class: 1



In [13]:
r = export_text(treeclf2, feature_names=feature_cols)
>>> print(r)

|--- Education <= 0.50
|   |--- ApplicantIncome <= 1810.00
|   |   |--- Dependents <= 1.00
|   |   |   |--- class: 0
|   |   |--- Dependents >  1.00
|   |   |   |--- class: 1
|   |--- ApplicantIncome >  1810.00
|   |   |--- ApplicantIncome <= 2267.00
|   |   |   |--- Married <= 0.50
|   |   |   |   |--- Dependents <= 1.50
|   |   |   |   |   |--- Gender <= 0.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- Gender >  0.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- Dependents >  1.50
|   |   |   |   |   |--- class: 0
|   |   |   |--- Married >  0.50
|   |   |   |   |--- Gender <= 0.50
|   |   |   |   |   |--- ApplicantIncome <= 2038.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- ApplicantIncome >  2038.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- Gender >  0.50
|   |   |   |   |   |--- class: 1
|   |   |--- ApplicantIncome >  2267.00
|   |   |   |--- ApplicantIncome <= 3078.50
|   |   |   |   |--- Dependents <= 1.50
|  